In [1]:
import re
import pandas as pd

import loggedfs # Python pass-through filesystem: pip install loggedfs

In [2]:
demo_filter = loggedfs.filter_pipeline_class(include_list = [loggedfs.filter_item_class([loggedfs.filter_field_class(
        name = 'proc_cmd', value = re.compile('.*kate.*').match
        )])]) # filter pipeline for filesystem actions coming from commands containing the string "kate" (the KDE text editor)

demo_data, demo_err = [], [] # dump logging data here

demo = loggedfs.loggedfs_notify(
    'demo_dir', # relative path to mountpoint
    background = True, # log in background thread
    log_filter = demo_filter, # apply filter pipeline
    log_only_modify_operations = True, # only log operations that have the potential to modify the filesystem
    log_buffers = True, # include read and write buffers into log
    consumer_out_func = demo_data.append, # consume logging data
    consumer_err_func = demo_err.append # consume potential errors
    ) # MOUNT (and start logging)

In [3]:
demo.terminate() # UMOUNT (and stop logging)

assert isinstance(demo_data[-1], loggedfs.end_of_transmission)
demo_data = demo_data[:-1]

len(demo_data)

27

In [4]:
data_df = pd.DataFrame.from_records(demo_data, index = 'time')

data_df[data_df['action'] != 'write'][['action', 'proc_pid', 'param_path', 'param_old_path', 'param_new_path', 'param_fip', 'param_buf_len', 'param_offset', 'param_mode', 'return', 'status']]

,action,proc_pid,param_path,param_old_path,param_new_path,param_fip,param_buf_len,param_offset,param_mode,return,status
time,,,,,,,,,,,
1556736520813215826,mknod,30977,/tmp/ernst/demo_dir/.demo_file.txt.kate-swp,NaN,NaN,NaN,NaN,NaN,33188.0,NaN,True
1556736520814149530,chmod,30977,/tmp/ernst/demo_dir/.demo_file.txt.kate-swp,NaN,NaN,NaN,NaN,NaN,33152.0,NaN,True
1556736524870363873,mknod,30977,/tmp/ernst/demo_dir/demo_file.txt.X30977,NaN,NaN,NaN,NaN,NaN,33152.0,NaN,True
1556736524871304006,chmod,30977,/tmp/ernst/demo_dir/demo_file.txt.X30977,NaN,NaN,NaN,NaN,NaN,33188.0,NaN,True
1556736524872197836,rename,30977,NaN,/tmp/ernst/demo_dir/demo_file.txt.X30977,/tmp/ernst/demo_dir/demo_file.txt,NaN,NaN,NaN,NaN,NaN,True
1556736524872611868,unlink,30977,/tmp/ernst/demo_dir/.demo_file.txt.kate-swp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
